# Model Based System Engineering Analyzer: 
## Leveraging AI for System Engineering

Requirement Document Review for Model Analysis


This notebook will load the requirements exported from teamcenter and use it to create a operation and system model. 


### Model-Specific Code (Do Not Modify)

This section contains code that is specific to the system model. It is updated only when the model is changed and should not require user modifications under normal circumstances.

If a new model is introduced, ensure this section is reviewed and updated as needed.


In [1]:
#!pip install --upgrade git+https://github.com/tkSDISW/Capella_Tools 
import capellambse.decl

from capella_tools import capellambse_helper


from IPython import display as diag_display
resources = {
    "EBSD_300": "EBSD_300/EBSD_300",
}
path_to_model = "../EBSD_300.aird"
model = capellambse.MelodyModel(path_to_model, resources=resources)





## 🎯 Prompt for System Model Element Analysis

### Purpose
This cell accepts a prompt to identify and isolate the specific object(s) to be analyzed. Providing detailed and accurate information in the prompt will ensure the best performance during the analysis.

### What to Include in Your Prompt
For optimal results, specify the following details about the object(s):
1. **Type of ARCADIA Object:** Clearly state the type of the object (e.g., Logical Component, Physical Component, Functional Exchange).
2. **Name of the Object:** Provide the exact name of the object as defined in the model.
3. **ARCADIA Phase:** Specify the phase associated with the object (e.g., Operational Analysis, System Analysis, Logical Architecture, Physical Architecture).
4. **Related Objects:** Identify any objects connected via exchanges or dependencies.

### Example Prompt
Locate the Logical Component named Brake Pedal in the Logical phase and has a Driver supplying input.
### ✅ Tips for Success
- **Use precise terminology:** Ensure your prompt aligns with the model's structure and terminology.
- **Include related objects:** Whenever possible, specify related objects to enhance context and improve analysis accuracy.
- **Double-check details:** Verify object names and phases to ensure they match the model exactly.
'''

### 🚀 Next Steps
1. Run the cell below.
2. Enter your prompt, providing the necessary details as outlined above.
3. Wait for the analysis to complete.
4. Select the best index or indexes for analysis. (The last selection will drive selection of a prompt.)

> 💡 **Tip:** If you're unsure about the model structure, review the documentation or refer to the model's diagrams for additional guidance.


In [2]:

from capella_tools  import capella_embeddings_manager

# Generate embeddings for all objects
model_embedding_manager = capella_embeddings_manager.EmbeddingManager()

embedding_file = "embeddings.json" 
model_embedding_manager.set_files( path_to_model , embedding_file)

model_embedding_manager.create_model_embeddings(model)

✅ EmbeddingManager initialized
🔐 API Key: Loaded from secrets
🌐 Base URL: https://api.openai.com/v1
🤖 Model: gpt-4o
Loading embeddings
embeddings loaded.


In [3]:
selected_objects = model_embedding_manager.query_and_select_top_objects("OCB] Operational Capabilities", top_n=1)
#Workflow
from capella_tools import capellambse_yaml_manager
yaml_handler = capellambse_yaml_manager.CapellaYAMLHandler()
   
#Generate YAML for the logical component and append to the file
for object in  selected_objects : 
    yaml_handler.generate_yaml(model.by_uuid(object["uuid"]))  


#yaml_handler.display()
yaml_handler.generate_yaml_referenced_objects()
#yaml_handler.display()

yaml_handler.write_output_file()


This is a list of ranked Objects Based on Query:
Index: 0, Name: [OCB] Operational Capabilities, Similarity: 0.78, Type: Diagram, Phase: Operational Analysis OA, Source: , Target: 


In [4]:
import os
from openai import OpenAI
from IPython.core.display import HTML
from IPython.display import display, clear_output, Markdown
from capella_tools  import Open_AI_RAG_manager


     


# Step 1: Get YAML content
yaml_content = yaml_handler.get_yaml_content()  


# Step 2: Invoke ChatGPT for analysis
analyzer = Open_AI_RAG_manager.ChatGPTAnalyzer(yaml_content)

analyzer.add_text_file_to_messages("how_to_book.pdf")
analyzer.add_text_file_to_messages("EBSD 300 Product Requirements Specification.pdf")


prompt = """
Generate a set of 40 system requirements for the Control Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 
"""

analyzer.initial_prompt(prompt)


chatgpt_response = analyzer.get_response()


✅ ChatGPTAnalyzer initialized
🔐 API Key: Loaded from secrets
🌐 Base URL: https://api.openai.com/v1
🤖 Model: gpt-4o
✅ File `how_to_book.pdf` added to messages for analysis.
✅ File `EBSD 300 Product Requirements Specification.pdf` added to messages for analysis.


**Your prompt:** 
Generate a set of 40 system requirements for the Control Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 
 Format the response in .html format.

**Token Usage Info:**

Tokens used: prompt=24110, completion=2139, total=26249

## ⚙️ Execute Followup Prompts

### Purpose
This cell executes the **follow up prompts** defined for analyzing the model. These prompts can be added by the note book user in addition to the default prompts

### What This Cell Does
- Use the structured input file (e.g., `capella_model.yaml`).
- Excutes the prompt defined in the cell.
- Outputs the results directly in the notebook.

### 🛠️ Instructions
1. Ensure the `capella_model.yaml` file is generated and up-to-date.
2. Run this cell to execute the default prompts.
3. Review the output to verify the accuracy and relevance of the analysis.

> 💡 **Tip:** Modify the promt to your needs.

In [5]:
prompt = """
Generate a set of 20 system functional requirements for Control  Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 
"""




analyzer.follow_up_prompt(prompt)
chatgpt_response = analyzer.get_response()

prompt = """
Generate a set of 20 system functional requirements for the Thermal Current Sensing Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 
"""




analyzer.follow_up_prompt(prompt)
chatgpt_response = analyzer.get_response()


**Your prompt:** 
Generate a set of 20 system functional requirements for Control  Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 


**Token Usage Info:**

Tokens used: prompt=26431, completion=1061, total=27492

**Your prompt:** 
Generate a set of 20 system functional requirements for the Thermal Current Sensing Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 


**Token Usage Info:**

Tokens used: prompt=27677, completion=1015, total=28692

In [6]:



prompt = """
Generate a set of 20 system functional requirements for the Power Driver Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 
"""



analyzer.follow_up_prompt(prompt)
chatgpt_response = analyzer.get_response()



prompt = """
Generate a set of 20 system functional requirements for the Chassis Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 
"""





analyzer.follow_up_prompt(prompt)
chatgpt_response = analyzer.get_response()


prompt = """
Generate a set of 20 system functional requirements for Power Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading fromat <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 
"""




analyzer.follow_up_prompt(prompt)
chatgpt_response = analyzer.get_response()


**Token Usage Info:**

Tokens used: prompt=28696, completion=1015, total=29711

**Your prompt:** 
Generate a set of 20 system functional requirements for the Power Driver Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 


**Token Usage Info:**

Tokens used: prompt=29894, completion=1005, total=30899

**Your prompt:** 
Generate a set of 20 system functional requirements for the Chassis Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 


**Token Usage Info:**

Tokens used: prompt=31082, completion=1064, total=32146

**Your prompt:** 
Generate a set of 20 system functional requirements for Power Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading fromat <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 


**Token Usage Info:**

Tokens used: prompt=32324, completion=984, total=33308

In [8]:
prompt = """
Generate a set of 20 system functional requirements for the General IO Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 
"""

analyzer.follow_up_prompt(prompt)
chatgpt_response = analyzer.get_response()

**Your prompt:** 
Generate a set of 20 system functional requirements for the General IO Subsystem using the yaml and documents for reference do not stop for brevity, only if no new requirements can be noted.
- Each requirment should be a tuple: (name, body, rationale).
- The name shall be five or fewer words introducing the requirement.
- The body shall be in the format of a shall statement, that is clear and concise. 
- The rationale should explain the source behind numeric requirement. 
- The response shall be formatted for cutting an pasting in to MS Word.
- The name should be a heading format with no preceding number <generate heading>
- The body and rationale should be in a paragraph with format <generated body text> Rationale: <generated rationale>
Use yaml content and attached pdf documents to provide context to the requirements for this subsystem. 


**Token Usage Info:**

Tokens used: prompt=33491, completion=1045, total=34536

# 